In [1]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from tqdm.auto import tqdm

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# !pip install Cython

In [4]:
# !pip install scikit-learn==0.22.2.post1 --upgrade

In [5]:
import os
import sys
import time
import math
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.data as data
import sklearn
import scipy
import collections
from scipy.linalg import norm
from PIL import Image
from sklearn.metrics import mean_squared_error
import json
#from scipy.optimize import linear_sum_assignment as linear_assignment

def weights_xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight.data)
        nn.init.constant(m.bias.data, 0)

def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    #from scipy.optimize import linear_sum_assignment as linear_assignment
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

class Dataset(data.Dataset):
    def __init__(self, data, labels, transform=None, target_transform=None):
        self.transform = transform
        self.target_transform = target_transform
        self.data = data
        self.labels = labels
        if torch.cuda.is_available():
            self.data = self.data.cuda()
            self.labels = self.labels.cuda()

    def __getitem__(self, index):
        img, target = self.data[index], self.labels[index]
        # img = Image.fromarray(img)
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.autograd import Variable
import math
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.cluster import KMeans


def buildNetwork(layers, activation="relu", dropout=0):
    net = []
    for i in range(1, len(layers)):
        net.append(nn.Linear(layers[i-1], layers[i]))
        if activation=="relu":
            net.append(nn.ReLU())
        elif activation=="sigmoid":
            net.append(nn.Sigmoid())
        if dropout > 0:
            net.append(nn.Dropout(dropout))
    return nn.Sequential(*net)



class IDEC(nn.Module):
    def __init__(self, input_dim=784, z_dim=10, n_clusters=10,
        encodeLayer=[400], decodeLayer=[400], activation="relu", dropout=0, alpha=1., gamma=0.1):
        super(self.__class__, self).__init__()
        self.z_dim = z_dim
        self.layers = [input_dim] + encodeLayer + [z_dim]
        self.activation = activation
        self.dropout = dropout
        self.encoder = buildNetwork([input_dim] + encodeLayer, activation=activation, dropout=dropout)
        self.decoder = buildNetwork([z_dim] + decodeLayer, activation=activation, dropout=dropout)
        self._enc_mu = nn.Linear(encodeLayer[-1], z_dim)
        self._dec = nn.Linear(decodeLayer[-1], input_dim)

        self.n_clusters = n_clusters
        self.alpha = alpha
        self.gamma = gamma
        self.mu = Parameter(torch.Tensor(n_clusters, z_dim))

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        pretrained_dict = torch.load(path, map_location=lambda storage, loc: storage)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict) 
        self.load_state_dict(model_dict)

    def forward(self, x):
        h = self.encoder(x)
        z = self._enc_mu(h)
        h = self.decoder(z)
        xrecon = self._dec(h)
        # compute q -> NxK
        q = self.soft_assign(z)
        return z, q, xrecon

    def soft_assign(self, z):
        q = 1.0 / (1.0 + torch.sum((z.unsqueeze(1) - self.mu)**2, dim=2) / self.alpha)
        q = q**(self.alpha+1.0)/2.0
        q = q / torch.sum(q, dim=1, keepdim=True)
        return q

    def encodeBatch(self, X, batch_size=256):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        
        encoded = []
        self.eval()
        num = X.shape[0]
        num_batch = int(math.ceil(1.0*X.shape[0]/batch_size))
        for batch_idx in range(num_batch):
            xbatch = X[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
            inputs = Variable(xbatch)
            z,_, _ = self.forward(inputs)
            encoded.append(z.data)

        encoded = torch.cat(encoded, dim=0)
        return encoded

    def cluster_loss(self, p, q):
        def kld(target, pred):
            return torch.mean(torch.sum(target*torch.log(target/(pred+1e-6)), dim=1))
        kldloss = kld(p, q)
        return self.gamma*kldloss

    def recon_loss(self, x, xrecon):
        recon_loss = torch.mean((xrecon-x)**2)
        return recon_loss

    def pairwise_loss(self, p1, p2, cons_type):
        if cons_type == "ML":
            ml_loss = torch.mean(-torch.log(torch.sum(p1 * p2, dim=1)))
            return ml_loss
        else:
            cl_loss = torch.mean(-torch.log(1.0 - torch.sum(p1 * p2, dim=1)))
            return cl_loss

    def global_size_loss(self, p, cons_detail):
        m_p = torch.mean(p, dim=0)
        m_p = m_p / torch.sum(m_p)
        return torch.sum((m_p-cons_detail)*(m_p-cons_detail))

    def difficulty_loss(self, q, mask):
        mask = mask.unsqueeze_(-1)
        mask = mask.expand(q.shape[0], q.shape[1])
        mask_q = q * mask
        diff_loss = -torch.norm(mask_q, 2)
        penalty_degree = 0.1
        return penalty_degree * diff_loss

    def target_distribution(self, q):
        p = q**2 / torch.sum(q, dim=0)
        p = p / torch.sum(p, dim=1, keepdim=True)
        return p

    def triplet_loss(self, anchor, positive, negative, margin_constant):
        # loss = max(d(anchor, negative) - d(anchor, positve) + margin, 0), margin > 0
        # d(x, y) = q(x) * q(y)
        negative_dis = torch.sum(anchor * negative, dim=1)
        positive_dis = torch.sum(anchor * positive, dim=1)
        margin = margin_constant * torch.ones(negative_dis.shape).cuda()
        diff_dis = negative_dis - positive_dis
        penalty = diff_dis + margin
        triplet_loss = 1*torch.max(penalty, torch.zeros(negative_dis.shape).cuda())

        return torch.mean(triplet_loss)

    def satisfied_constraints(self,ml_ind1,ml_ind2,cl_ind1, cl_ind2,y_pred):
        
        if ml_ind1.size == 0 or ml_ind2.size == 0 or cl_ind1.size == 0 or cl_ind2.size == 0:
            return 1.0

        count = 0
        satisfied = 0
        for (i, j) in zip(ml_ind1, ml_ind2):
            count += 1
            if y_pred[i] == y_pred[j]:
                satisfied += 1
        for (i, j) in zip(cl_ind1, cl_ind2):
            count += 1
            if y_pred[i] != y_pred[j]:
                satisfied += 1

        return float(satisfied)/count


    def predict(self, X, y):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        latent = self.encodeBatch(X)
        q = self.soft_assign(latent)

        # evalute the clustering performance
        y_pred = torch.argmax(q, dim=1).data.cpu().numpy()
        y = y.data.cpu().numpy()
        if y is not None:
            print("acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
            final_acc = acc(y, y_pred)
            final_nmi = normalized_mutual_info_score(y, y_pred)
        return final_acc, final_nmi, y_pred

    def fit(self,anchor, positive, negative, ml_ind1,ml_ind2,cl_ind1, cl_ind2, mask, use_global, ml_p, cl_p, X,y=None, lr=0.001, batch_size=256, num_epochs=10, update_interval=1, tol=1e-3, use_kmeans=True, plotting="",clustering_loss_weight=1):    
        
        # save intermediate results for plotting
        intermediate_results = collections.defaultdict(lambda:{})
        
        '''X: tensor data'''
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.cuda()
        print("=====Training IDEC=======")
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.parameters()), lr=lr)

        if use_kmeans:
            print("Initializing cluster centers with kmeans.")
            kmeans = KMeans(self.n_clusters, n_init=20)
            data = self.encodeBatch(X)
            y_pred = kmeans.fit_predict(data.data.cpu().numpy())
            y_pred_last = y_pred
            self.mu.data.copy_(torch.Tensor(kmeans.cluster_centers_))
        else:
            # use kmeans to randomly initialize cluster ceters
            print("Randomly initializing cluster centers.")
            kmeans = KMeans(self.n_clusters, n_init=1, max_iter=1)
            data = self.encodeBatch(X)
            y_pred = kmeans.fit_predict(data.data.cpu().numpy())
            y_pred_last = y_pred
            self.mu.data.copy_(torch.Tensor(kmeans.cluster_centers_))

        if y is not None:
            y = y.cpu().numpy()
            # print("Kmeans acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
        self.train()
        num = X.shape[0]
        num_batch = int(math.ceil(1.0*X.shape[0]/batch_size))
        ml_num_batch = int(math.ceil(1.0*ml_ind1.shape[0]/batch_size))
        cl_num_batch = int(math.ceil(1.0*cl_ind1.shape[0]/batch_size))
        tri_num_batch = int(math.ceil(1.0*anchor.shape[0]/batch_size))
        cl_num = cl_ind1.shape[0]
        ml_num = ml_ind1.shape[0]
        tri_num = anchor.shape[0]

        final_acc, final_nmi, final_epoch = 0, 0, 0
        update_ml = 1
        update_cl = 1
        update_triplet = 1
        for epoch in range(num_epochs):
            if epoch%update_interval == 0:
                # update the targe distribution p
                latent = self.encodeBatch(X)
                q = self.soft_assign(latent)
                p = self.target_distribution(q).data

                # evalute the clustering performance
                y_pred = torch.argmax(q, dim=1).data.cpu().numpy()
                if use_global:
                    y_dict = collections.defaultdict(list)
                    ind1, ind2 = [], []
                    for i in range(y_pred.shape[0]):
                        y_dict[y_pred[i]].append(i)
                    for key in y_dict.keys():
                        if y is not None:
                            print("predicted class: ", key, " total: ", len(y_dict[key]))
                            #, " mapped index(ground truth): ", np.bincount(y[y_dict[key]]).argmax())

                if y is not None:
                    print("acc: %.5f, nmi: %.5f" % (acc(y, y_pred), normalized_mutual_info_score(y, y_pred)))
                    print("satisfied constraints: %.5f"%self.satisfied_constraints(ml_ind1,ml_ind2,cl_ind1, cl_ind2,y_pred))
                    final_acc = acc(y, y_pred)
                    final_nmi = normalized_mutual_info_score(y, y_pred)
                    final_epoch = epoch

                # save model for plotting
                if plotting and (epoch in [10,20,30,40] or epoch%50 == 0 or epoch == num_epochs-1):
                    
                    df = pd.DataFrame(latent.cpu().numpy())
                    df["y"] = y
                    df.to_pickle(os.path.join(plotting,"save_model_%d.pkl"%(epoch)))
                    
                    intermediate_results["acc"][str(epoch)] = acc(y, y_pred)
                    intermediate_results["nmi"][str(epoch)] = normalized_mutual_info_score(y, y_pred)
                    with open(os.path.join(plotting,"intermediate_results.json"), "w") as fp:
                        json.dump(intermediate_results, fp)

                # check stop criterion
                try:
                    delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / num
                    y_pred_last = y_pred
                    if epoch>0 and delta_label < tol:
                        print('delta_label ', delta_label, '< tol ', tol)
                        print("Reach tolerance threshold. Stopping training.")

                        # save model for plotting
                        if plotting:
                            
                            df = pd.DataFrame(latent.cpu().numpy())
                            df["y"] = y
                            df.to_pickle(os.path.join(plotting,"save_model_%d.pkl"%epoch))
                            
                            intermediate_results["acc"][str(epoch)] = acc(y, y_pred)
                            intermediate_results["nmi"][str(epoch)] = normalized_mutual_info_score(y, y_pred)
                            with open(os.path.join(plotting,"intermediate_results.json"), "w") as fp:
                                json.dump(intermediate_results, fp)
                        break
                except:
                    pass

            # train 1 epoch for clustering loss
            train_loss = 0.0
            recon_loss_val = 0.0
            cluster_loss_val = 0.0
            instance_constraints_loss_val = 0.0
            global_loss_val = 0.0
            for batch_idx in range(num_batch):
                xbatch = X[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                pbatch = p[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                mask_batch = mask[batch_idx*batch_size : min((batch_idx+1)*batch_size, num)]
                optimizer.zero_grad()
                inputs = Variable(xbatch)
                target = Variable(pbatch)
                cons_detail = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
                global_cons = torch.from_numpy(cons_detail).float().to("cuda")

                z, qbatch, xrecon = self.forward(inputs)
                if use_global == False:
                    cluster_loss = self.cluster_loss(target, qbatch)
                    recon_loss = self.recon_loss(inputs, xrecon)
                    instance_constraints_loss = self.difficulty_loss(qbatch, mask_batch)
                    loss = cluster_loss + recon_loss + instance_constraints_loss
                    loss.backward()
                    optimizer.step()
                    cluster_loss_val += cluster_loss.data * len(inputs)
                    recon_loss_val += recon_loss.data * len(inputs)
                    instance_constraints_loss_val += instance_constraints_loss.data * len(inputs)
                    train_loss = clustering_loss_weight*cluster_loss_val + recon_loss_val + instance_constraints_loss_val
                else:
                    cluster_loss = self.cluster_loss(target, qbatch)
                    recon_loss = self.recon_loss(inputs, xrecon)
                    global_loss = self.global_size_loss(qbatch, global_cons)
                    loss = cluster_loss + recon_loss + global_loss
                    loss.backward()
                    optimizer.step()
                    cluster_loss_val += cluster_loss.data * len(inputs)
                    recon_loss_val += recon_loss.data * len(inputs)
                    train_loss = clustering_loss_weight*cluster_loss_val + recon_loss_val


            if instance_constraints_loss_val != 0.0:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f Instance Difficulty Loss: %.4f"% (
                    epoch + 1, train_loss / num, cluster_loss_val / num, recon_loss_val / num, instance_constraints_loss_val / num))
            elif global_loss_val != 0.0 and use_global:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f Global Loss: %.4f"% (
                    epoch + 1, train_loss / num + global_loss_val/num_batch, cluster_loss_val / num, recon_loss_val / num, global_loss_val / num_batch))
            else:
                print("#Epoch %3d: Total: %.4f Clustering Loss: %.4f Reconstruction Loss: %.4f" % (
                    epoch + 1, train_loss / num, cluster_loss_val / num, recon_loss_val / num))
            ml_loss = 0.0
            if epoch % update_ml == 0:
                for ml_batch_idx in range(ml_num_batch):
                    px1 = X[ml_ind1[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    px2 = X[ml_ind2[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    pbatch1 = p[ml_ind1[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[ml_ind2[ml_batch_idx*batch_size : min(ml_num, (ml_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    loss = (ml_p*self.pairwise_loss(q1, q2, "ML")+self.recon_loss(inputs1, xr1) + self.recon_loss(inputs2, xr2))
                    # 0.1 for mnist/reuters, 1 for fashion, the parameters are tuned via grid search on validation set
                    ml_loss += loss.data
                    loss.backward()
                    optimizer.step()

            cl_loss = 0.0
            if epoch % update_cl == 0:
                for cl_batch_idx in range(cl_num_batch):
                    px1 = X[cl_ind1[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    px2 = X[cl_ind2[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    pbatch1 = p[cl_ind1[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[cl_ind2[cl_batch_idx*batch_size : min(cl_num, (cl_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    loss = cl_p*self.pairwise_loss(q1, q2, "CL")
                    cl_loss += loss.data
                    loss.backward()
                    optimizer.step()

            if ml_num_batch >0 and cl_num_batch > 0:
                print("Pairwise Total:", round(float(ml_loss.cpu()), 2) + float(cl_loss.cpu()), "ML loss", float(ml_loss.cpu()), "CL loss:", float(cl_loss.cpu()))
            triplet_loss = 0.0
            if epoch % update_triplet == 0:
                for tri_batch_idx in range(tri_num_batch):
                    px1 = X[anchor[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    px2 = X[positive[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    px3 = X[negative[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    pbatch1 = p[anchor[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx + 1)*batch_size)]]
                    pbatch2 = p[positive[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    pbatch3 = p[negative[tri_batch_idx*batch_size : min(tri_num, (tri_batch_idx+1)*batch_size)]]
                    optimizer.zero_grad()
                    inputs1 = Variable(px1)
                    inputs2 = Variable(px2)
                    inputs3 = Variable(px3)
                    target1 = Variable(pbatch1)
                    target2 = Variable(pbatch2)
                    target3 = Variable(pbatch3)
                    z1, q1, xr1 = self.forward(inputs1)
                    z2, q2, xr2 = self.forward(inputs2)
                    z3, q3, xr3 = self.forward(inputs3)
                    loss = self.triplet_loss(q1, q2, q3, 0.1)
                    triplet_loss += loss.data
                    loss.backward()
                    optimizer.step()
            if tri_num_batch > 0:
                print("Triplet Loss:", triplet_loss)
        return final_acc, final_nmi, final_epoch

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by 

In [6]:
print(sklearn.__version__)

0.21.3


In [7]:
import os
import os.path
import errno
import numpy as np
import gzip
import torch
import pickle
import torch.utils.data as data
import codecs
import urllib
class Netflix(data.Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.use_cuda = torch.cuda.is_available()
        self.training_file = "users.npy" # your data for clustering (e.g., user-predictions matrix, user-embedding learned from SVD ...)
        self.label_file = "clusters.npy" # clustering labels (not applicable for our assignment, I used the k-means labels for svd_pu file)
        if download:
            self.download()
        train = np.load(self.training_file)
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        label = np.load(self.label_file)
        train = scaler.fit_transform(train)
        if self.train:
            self.train_data, self.train_labels = torch.tensor(train, dtype=torch.float32), torch.tensor(label, dtype=torch.int)
            if self.use_cuda:
                self.train_data = self.train_data.cuda()
                self.train_labels = self.train_labels.cuda()
        else:
            self.test_data, self.test_labels = torch.tensor(train, dtype=torch.float32), torch.tensor(label, dtype=torch.int)
            if self.use_cuda:
                self.test_data = self.test_data.cuda()
                self.test_labels = self.test_labels.cuda()
    def __getitem__(self, index):
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        return img, target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

In [8]:
#Run_IDEC
netflix_train = Netflix('', train=True)
netflix_test = Netflix('', train=False) # here the test and train are the same
X, y = netflix_train.train_data, netflix_train.train_labels
test_X, test_y = netflix_train.train_data, netflix_train.train_labels

idec = IDEC(input_dim=15895, z_dim=20, n_clusters=3,
           encodeLayer=[500, 500, 2000], decodeLayer=[2000, 500, 500], activation="relu", dropout=0)
# Print network structure
print(idec)

# Set parameters
lr = 0.001
batch_size = 512
epochs = 100
update_interval = 1
ml_penalty, cl_penalty = 0.1, 1
# Construct constraints (here is the baseline so no constraints are provided).
ml_ind1, ml_ind2, cl_ind1, cl_ind2 = np.array([]), np.array([]), np.array([]), np.array([])
anchor, positive, negative = np.array([]), np.array([]), np.array([])
instance_guidance = torch.zeros(X.shape[0]).cuda()
use_global = False
idec.load_model('sdae_netflix_weights.pt') # load your pre-trained model from SDAE.ipynb
# Train the clustering model
train_acc, train_nmi, epo = idec.fit(anchor, positive, negative, ml_ind1, ml_ind2, cl_ind1, cl_ind2, instance_guidance, use_global,  ml_penalty, cl_penalty, X, y,
                          lr=lr, batch_size=batch_size, num_epochs=epochs,update_interval=update_interval,tol=1*1e-4, clustering_loss_weight=1)

IDEC(
  (encoder): Sequential(
    (0): Linear(in_features=15895, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=2000, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
  )
  (_enc_mu): Linear(in_features=2000, out_features=20, bias=True)
  (_dec): Linear(in_features=500, out_features=15895, bias=True)
)
=====Training IDEC=======
Initializing cluster centers with kmeans.


/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/cluster/k_means_.py:443: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  centers, labels, n_iter = k_means_elkan(X, checked_sample_weight,
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/cluster/k_means_.py:443: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify t

acc: 0.47417, nmi: 0.28345
satisfied constraints: 1.00000
#Epoch   1: Total: 0.7346 Clustering Loss: 0.0688 Reconstruction Loss: 0.6657
acc: 0.34610, nmi: 0.08505
satisfied constraints: 1.00000


/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/metrics/cluster/supervised.py:123: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:123: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:93: DeprecationWarning: `np.bo

#Epoch   2: Total: 0.3682 Clustering Loss: 0.0055 Reconstruction Loss: 0.3627
acc: 0.36507, nmi: 0.12348
satisfied constraints: 1.00000
#Epoch   3: Total: 0.3239 Clustering Loss: 0.0060 Reconstruction Loss: 0.3178
acc: 0.37727, nmi: 0.13361
satisfied constraints: 1.00000


...

#Epoch  96: Total: 0.1824 Clustering Loss: 0.0037 Reconstruction Loss: 0.1787
acc: 0.40979, nmi: 0.16299
satisfied constraints: 1.00000
#Epoch  97: Total: 0.1825 Clustering Loss: 0.0036 Reconstruction Loss: 0.1788
acc: 0.41030, nmi: 0.16328
satisfied constraints: 1.00000


/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:123: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.row_uncovered = np.ones(n, dtype=np.bool)
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:94: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itse

#Epoch  98: Total: 0.1831 Clustering Loss: 0.0034 Reconstruction Loss: 0.1797
acc: 0.41030, nmi: 0.16316
satisfied constraints: 1.00000
#Epoch  99: Total: 0.1858 Clustering Loss: 0.0040 Reconstruction Loss: 0.1817
acc: 0.41047, nmi: 0.16366
satisfied constraints: 1.00000


/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:123: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.row_uncovered = np.ones(n, dtype=np.bool)
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:94: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itse

#Epoch 100: Total: 0.1855 Clustering Loss: 0.0039 Reconstruction Loss: 0.1817


In [9]:
# Generate predictions on the test data
test_acc, test_nmi, y_pred = idec.predict(test_X, test_y)
np.save("idec_pred.npy", y_pred)

acc: 0.41081, nmi: 0.16405


/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:123: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:93: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.row_uncovered = np.ones(n, dtype=np.bool)
/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/linear_assignment_.py:94: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itse

In [10]:
print(y_pred.max())

2


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
y_pred = np.load("idec_pred.npy")

df = pd.read_csv("train.csv", sep=",")
df.head
training_data1 = pd.read_csv("training.csv", sep=",")
validation_data = pd.read_csv("validation.csv", sep=",")
# training_data1, validation_data = train_test_split(df, random_state=10, test_size=0.1)



user_X_movie = training_data1.pivot_table(index='customer-id', columns='movie-id', values='rating')
user_X_movie_zeros = user_X_movie.fillna(0)
# df_p_nan = training_data.pivot_table(index='customer-id', columns='movie-id', values='rating')
# df_p = df_p_nan.fillna(np.nan)
# print(df_p.head)
number_of_clusters = 3

In [12]:
labels = y_pred
print("\n\CLUSTERS BUILT\n\n")
import pandas as pd
cluster_map = pd.DataFrame()
cluster_map['customer_id'] = user_X_movie_zeros.index.values
cluster_map['cluster_no'] = labels


\CLUSTERS BUILT




In [13]:
cluster_map.rename(columns = {'customer_id':'customer-id'}, inplace = True)

print("Saving cluster map")
cluster_map.to_csv('clusterMap.csv')

Saving cluster map


In [14]:
training_data_with_labels = pd.merge(training_data1, cluster_map, on='customer-id')

In [15]:
for i in range(number_of_clusters):
    rows_from_same_cluster = training_data_with_labels[training_data_with_labels['cluster_no']==i]
    rows_from_same_cluster.to_csv('clustered_records_'+str(i)+'.csv',index=False)

In [16]:
print("\n\nCREATING CSVs\n\n")
for i in range(number_of_clusters):
    clustered_records = pd.read_csv("clustered_records_"+str(i)+".csv")
    unique_m_ids = clustered_records['movie-id'].unique()
    clustered_records['mean_rating'] = 0
    
    for mid in unique_m_ids:
        mean_rating = round(clustered_records[clustered_records['movie-id']==mid]['rating'].mean()) #.round()
        clustered_records.loc[clustered_records['movie-id']==mid, 'mean_rating'] = mean_rating
        
    clustered_records['mean_rating'] = clustered_records['mean_rating'].astype(int)
    clustered_records.to_csv('clustered_records_'+str(i)+'.csv',index=False)

print("\n\nCREATED ALL CSVs\n\n")



CREATING CSVs




CREATED ALL CSVs




In [17]:
c0 = pd.read_csv("clustered_records_0.csv")
c1 = pd.read_csv("clustered_records_1.csv")
c2 = pd.read_csv("clustered_records_2.csv")
# c3 = pd.read_csv("clustered_records_3.csv")
# c4 = pd.read_csv("clustered_records_4.csv")
# c5 = pd.read_csv("clustered_records_5.csv")
# c6 = pd.read_csv("clustered_records_6.csv")
# c7 = pd.read_csv("clustered_records_7.csv")
# c8 = pd.read_csv("clustered_records_8.csv")
# c9 = pd.read_csv("clustered_records_9.csv")
# c10 = pd.read_csv("clustered_records_10.csv")
# c11 = pd.read_csv("clustered_records_11.csv")
# c12 = pd.read_csv("clustered_records_12.csv")
# c13 = pd.read_csv("clustered_records_13.csv")
# c14 = pd.read_csv("clustered_records_14.csv")
# c15 = pd.read_csv("clustered_records_15.csv")
# c16 = pd.read_csv("clustered_records_16.csv")
# c17 = pd.read_csv("clustered_records_17.csv")
# c18 = pd.read_csv("clustered_records_18.csv")
# c19 = pd.read_csv("clustered_records_19.csv")





final_search_cluster = c0.append([c1,c2]) #,c3,c4]) #,c5]) #,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19])
final_search_cluster.to_csv("final_search_cluster.csv")

print("\n\nSTARTING TESTs\n\n")

/tmp/ipykernel_3159009/331805596.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_search_cluster = c0.append([c1,c2]) #,c3,c4]) #,c5]) #,c6,c7,c8,c9,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19])




STARTING TESTs




In [18]:
# validation_data = testing_data
validation_data_clean_actual = pd.concat([validation_data['movie-id'], validation_data['customer-id'], validation_data['rating']], axis=1)
validation_data_clean_to_predict = pd.concat([validation_data['movie-id'], validation_data['customer-id'], validation_data['rating']], axis=1)

In [19]:
recno = 0
for index, row in tqdm(validation_data_clean_to_predict.iterrows()):
    recno+=1
    movie_id, customer_id = row['movie-id'], row['customer-id']
    try:
        cluster_number = cluster_map[cluster_map['customer-id'] == customer_id]['cluster_no'].iloc[0]
#        print("\t movie_id = ",movie_id, "\t customer_id = ",customer_id, "\t cluster_number = ",cluster_number)
        mean_rating = final_search_cluster[final_search_cluster['movie-id']==movie_id]['mean_rating'].iloc[0]
    except:
#         print("@@@@@@@@@@@ not in CLUSTER_MAP @@@@@@@@@@@")
#         print("\t movie_id = ",movie_id, "\t customer_id = ",customer_id, "\t cluster_number = ",cluster_number)
        mean_rating = 3

    # print("mean_rating = ",mean_rating, "\t #record = ",recno)
    validation_data_clean_to_predict.at[index,'mean_rating'] = mean_rating

validation_data_clean_to_predict.to_csv("validation_predictions.csv")

print("validation_data_clean_to_predict : \n",validation_data_clean_to_predict.head(5))

mse = mean_squared_error(validation_data_clean_actual['rating'], validation_data_clean_to_predict['mean_rating'])
print("MSE : ",mse)

100000it [02:08, 779.78it/s]


validation_data_clean_to_predict : 
    movie-id  customer-id  rating  mean_rating
0      7513        30165       4          4.0
1     16022         2878       5          4.0
2      9415        21820       1          3.0
3     14312        26109       2          4.0
4     12497         8382       5          3.0
MSE :  1.17912


In [20]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(validation_data_clean_actual['rating'], validation_data_clean_to_predict['mean_rating'])
print("MSE : ",mse)

MSE :  1.17912


In [21]:
test_df = pd.read_csv("test.csv", names=['movie_id','customer-id','rating', 'date'])

test_df['rating']=0
print(test_df.head)

recno = 0
for index, row in tqdm(test_df.iterrows()):
    recno+=1
    movie_id, customer_id = row['movie_id'], row['customer-id']
  
    
    try:
        cluster_number = cluster_map[cluster_map['customer-id'] == customer_id]['cluster_no'].iloc[0]
        mean_rating = final_search_cluster[final_search_cluster['movie-id']==movie_id]['mean_rating'].iloc[0]
       
        
    except:
        mean_rating = 3


    test_df.at[index,'mean_rating'] = mean_rating
    
print("test_df : \n",test_df.head(5))

test_df.to_csv("Deep_TEST_PREDICTIONS.csv")
test_rating = test_df['mean_rating'].tolist()
with open('Deep_prediction_results.txt', 'w') as file:
    for result in test_rating:
        file.write('%d\n' % result)

<bound method NDFrame.head of         movie_id  customer-id  rating        date
0          11089        11765       0  2005-06-03
1          11279         5858       0  2005-05-07
2          12140         5202       0  2004-01-24
3          12191        18818       0  2005-03-27
4          12299        10021       0  2005-09-07
...          ...          ...     ...         ...
249995     16459        11043       0  2003-10-15
249996     16535         7063       0  2003-02-12
249997     16707        32275       0  2004-07-29
249998     17339        10728       0  2005-07-15
249999     17580         5600       0  2005-06-18

[250000 rows x 4 columns]>


105446it [02:13, 789.19it/s]


KeyboardInterrupt: 